In [14]:
using Random, Statistics, Printf

function rosenbrock(x, y)
    return (1 - x)^2 + 100 * (y - x^2)^2
end

function schwefel(x, y)
    return 418.9829 * 2 - (x * sin(sqrt(abs(x))) + y * sin(sqrt(abs(y))))
end

function rastrigin(x, y)
    return 20 + x^2 - 10 * cos(2 * π * x) + y^2 - 10 * cos(2 * π * y)
end

function initialize_particles(num_particles, bounds)
    particles = [[rand(bounds[1]:0.01:bounds[2]), rand(bounds[1]:0.01:bounds[2])] for _ in 1:num_particles]
    velocities = [[rand(bounds[1]:0.01:bounds[2]), rand(bounds[1]:0.01:bounds[2])] for _ in 1:num_particles]
    return particles, velocities
end

function particle_swarm_optimization(num_particles, bounds, w, c1, c2, max_iters, objective_function, teleport=false)
    particles, velocities = initialize_particles(num_particles, bounds)
    personal_best = deepcopy(particles)
    personal_best_values = [objective_function(p[1], p[2]) for p in particles]

    global_best_index = argmin(personal_best_values)
    global_best = personal_best[global_best_index]
    global_best_value = personal_best_values[global_best_index]

    for iter in 1:max_iters
        for i in 1:num_particles
            velocities[i] .= w .* velocities[i] .+
                             c1 .* rand() .* (personal_best[i] .- particles[i]) .+
                             c2 .* rand() .* (global_best .- particles[i])

            particles[i] .= particles[i] .+ velocities[i]
            particles[i] .= clamp.(particles[i], bounds[1], bounds[2])

            value = objective_function(particles[i][1], particles[i][2])
            if value < personal_best_values[i]
                personal_best[i] = copy(particles[i])
                personal_best_values[i] = value
            end

            if value < global_best_value
                global_best = copy(particles[i])
                global_best_value = value
            end
        end

        if teleport && iter % 10 == 0
            for i in 1:num_particles
                if rand() < 0.1
                    particles[i] = [rand(bounds[1]:0.01:bounds[2]), rand(bounds[1]:0.01:bounds[2])]
                end
            end
        end
    end

    return global_best, global_best_value
end

function test_num_particles(objective_function, name, bounds)
    @printf("\n--- Зависимость от числа частиц (%s) ---\n", name)
    for num_particles in [10, 20, 50, 100, 200]
        start_time = time()
        best_point, best_value = particle_swarm_optimization(num_particles, bounds, 0.5, 1.5, 1.5, 100, objective_function)
        elapsed = time() - start_time
        @printf("Частиц: %d | Время: %.5f сек | Минимум: %.5f | Точка: (%.5f, %.5f)\n",
                num_particles, elapsed, best_value, best_point[1], best_point[2])
    end
end

function test_global_component(objective_function, name, bounds)
    @printf("\n--- Зависимость от c2 (%s) ---\n", name)
    for c2 in [0.5, 1.0, 1.5, 2.0, 2.5]
        start_time = time()
        best_point, best_value = particle_swarm_optimization(50, bounds, 0.5, 1.5, c2, 100, objective_function)
        elapsed = time() - start_time
        @printf("c2: %.1f | Время: %.5f сек | Минимум: %.5f | Точка: (%.5f, %.5f)\n",
                c2, elapsed, best_value, best_point[1], best_point[2])
    end
end

function compare_teleportation(objective_function, name, bounds)
    @printf("\n--- Сравнение телепортации (%s) ---\n", name)

    start_time = time()
    best_point_classic, best_value_classic = particle_swarm_optimization(50, bounds, 0.5, 1.5, 1.5, 100, objective_function)
    elapsed_classic = time() - start_time

    start_time = time()
    best_point_teleport, best_value_teleport = particle_swarm_optimization(50, bounds, 0.5, 1.5, 1.5, 100, objective_function, true)
    elapsed_teleport = time() - start_time
    @printf("Классический: Время: %.5f сек | Минимум: %.5f | Точка: (%.5f, %.5f)\n",
            elapsed_classic, best_value_classic, best_point_classic[1], best_point_classic[2])

    @printf("Телепортация: Время: %.5f сек | Минимум: %.5f | Точка: (%.5f, %.5f)\n",
            elapsed_teleport, best_value_teleport, best_point_teleport[1], best_point_teleport[2])
end

test_num_particles(rosenbrock, "Розенброк", (-2.0, 2.0))
test_global_component(rosenbrock, "Розенброк", (-2.0, 2.0))
compare_teleportation(rosenbrock, "Розенброк", (-2.0, 2.0))

test_num_particles(schwefel, "Швефель", (-500.0, 500.0))
test_global_component(schwefel, "Швефель", (-500.0, 500.0))
compare_teleportation(schwefel, "Швефель", (-500.0, 500.0))

test_num_particles(rastrigin, "Растригин", (-5.12, 5.12))
test_global_component(rastrigin, "Растригин", (-5.12, 5.12))
compare_teleportation(rastrigin, "Растригин", (-5.12, 5.12))


--- Зависимость от числа частиц (Розенброк) ---
Частиц: 10 | Время: 0.36700 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Частиц: 20 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Частиц: 50 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Частиц: 100 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Частиц: 200 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)

--- Зависимость от c2 (Розенброк) ---
c2: 0.5 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 1.0 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 1.5 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 2.0 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 2.5 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)

--- Сравнение телепортации (Розенброк) ---
Классический: Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Телепортаци